In [1]:
!pip install -U -q PyDrive
!apt-get install libopenblas-dev 

!conda create --name lightnet python=3  --yes
!source activate lightnet 
!pip install pathlib
!pip install numpy
!pip install plac
!pip install requests
!pip install msgpack-python
!pip install lightnet

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libopenblas-base
The following NEW packages will be installed:
  libopenblas-base libopenblas-dev
0 upgraded, 2 newly installed, 0 to remove and 0 not upgraded.
Need to get 7,824 kB of archives.
After this operation, 94.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu artful/universe amd64 libopenblas-base amd64 0.2.20+ds-4 [3,964 kB]
Get:2 http://archive.ubuntu.com/ubuntu artful/universe amd64 libopenblas-dev amd64 0.2.20+ds-4 [3,860 kB]
Fetched 7,824 kB in 1s (7,006 kB/s)
Selecting previously unselected package libopenblas-base:amd64.
(Reading database ... 16712 files and directories currently installed.)
Preparing to unpack .../libopenblas-base_0.2.20+ds-4_amd64.deb ...
Unpacking libopenblas-base:amd64 (0.2.20+ds-4) ...
Selecting previously unselected package libopenblas-dev:amd64.
Preparing to unpack

    100% |████████████████████████████████| 368kB 2.0MB/s 
    100% |████████████████████████████████| 51kB 6.1MB/s 
  Running setup.py bdist_wheel for lightnet ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /content/.cache/pip/wheels/db/68/ba/eb06d1292f9f16ec137792ae23582bf2bfc68f48531fae4e24
Successfully built lightnet


In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def getFile(file_name, target_id):

  # 1. Authenticate and create the PyDrive client.
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  #2. Get the file
  downloaded = drive.CreateFile({'id':target_id}) # replace the id with id of file you want to access
  downloaded.GetContentFile(file_name)  
  print 'done downloading '+ file_name

In [0]:
def uploadFileToDrive(nameOfFile):
      auth.authenticate_user()
      gauth = GoogleAuth()
      gauth.credentials = GoogleCredentials.get_application_default()
      drive = GoogleDrive(gauth)
      file = drive.CreateFile()
      file.SetContentFile(nameOfFile)
      file.Upload()

In [6]:
# TODO for test video
files_X = ['test_vid_1.avi', 'test_vid_2.avi']
files_Y = ['labels_1.csv', 'labels_2.csv']
files_X_key = ['1LwZBim4W_aAS52ADH_JNkMMwMlwvboKq', '1zAGCSpkjY5dQL86cCHjsw1OSnOo26J3Q']
files_Y_key = ['15YNRctoaFQCELUibW13sDgb-UDqwJZBm','1kMXEDc_Fb8WbiVy9el2Lm2zIKhX2sFU2']

for i in range(0, len(files_X)):
  getFile(files_X[i], files_X_key[i])
  getFile(files_Y[i], files_Y_key[i])

done downloading test_vid_1.avi
done downloading labels_1.csv
done downloading test_vid_2.avi
done downloading labels_2.csv


In [7]:
model_file = ['my_model.h5', 'my_model-noisefree.h5', 'my_model-noisefreeInception.h5']
model_file_key = ['1RyCOwl57SgIj9sZKhIQdhda0M4YAAGZV','1eptLBIJyNEWOpmYBebTZMX8iNR07HMLB', '1ZYPEqasvrMIpDSGUYmYQjWOS8OQYf9v0']
getFile(model_file[2], model_file_key[2])

done downloading my_model-noisefreeInception.h5


In [0]:
y_vals = {
    'SG':0,
    'FR':1,
    'AK':2,
    'SP':3,
    'SK':4,
    'SM':5,
    'jk':6
}

In [0]:
import numpy as np
import csv
y_list = list()
with open(files_Y[1]) as f:
  reader = csv.reader(f)
  for row in reader:
    s = ''
    for ab in row:
      s = s + ab
    y_list.append(y_vals[s])

y_list = np.array(y_list)

In [0]:
import cv2
import numpy as np
import PIL.Image
import matplotlib.pyplot as plt
from cStringIO import StringIO
import IPython.display
from tempfile import TemporaryFile
def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [11]:
!python -m lightnet download yolo

199kK [00:30, 6.59kK/s]                                                         


In [0]:
import lightnet
from lightnet import Image

model = lightnet.load('yolo')


In [0]:
def detectPerson(img):

  boxes = model(img)
#   print (boxes)
  return boxes

In [0]:
def cropImages(boxes, img):
  imageList = list()
  for box in boxes:
    
    if box[0] == 0:
      
      center_x = box[3][0]
      center_y = box[3][1]
      width = box[3][2]
      height = box[3][3]

      x = max(int(center_x - width/2), 0)
      y = max(int(center_y - height/2),0)
      width = int(width)
      height = int(height)
      # Crop!!
      crop_img = img[y:y+height, x:x+width]
#       showarray(crop_img)
      try:
        crop_img = cv2.resize(crop_img, (150, 120))
      except cv2.error as e:
        print x,y,width, height
        print 'could not crop'
        break
      
      imageList.append(crop_img)
      

    
  return imageList

In [0]:
image_shape = (300, 250)
def test(video_path, labels):
    vidcap = cv2.VideoCapture(video_path)
    correct = 0
    count = 0
    k = 0
    success = True
    imLis = list()
    # print vidcap
    y_labls = list()
    while success:
        
        success, image = vidcap.read()
        count += 1
        if count%20 == 0:
#           break
          print correct, count, k
#         img = cv2.resize(image, image_shape)
        img = image
        cv2.imwrite('frame1.jpg', img)
        im = lightnet.Image.from_bytes(open('frame1.jpg', 'rb').read())
        
        boxes = detectPerson(im)
        
        if len(boxes) == 0:
          if labels[count-1] == 6:
            correct = correct + 1
        continue
#         imList = np.array(imList, dtype=np.float32)
        imList = np.array(imList, dtype=np.float32)/255.0
#         print imList
        
        y = model_net.predict(imList)
        
        preds = np.argmax(y, axis=1)
#         print y, preds, labels[count-1]
#         print y
        flag = 0
        for pred in preds:
          if pred == labels[count-1]:
            correct += 1
            flag = 1
            break
#         if flag == 0:
#           showarray(imList[0]*255.0)
          
            
    acc = float(correct)/count
    print 'acc : ', acc


In [16]:
from keras.models import load_model

model_net = load_model('my_model-noisefreeInception.h5')

Using TensorFlow backend.


In [0]:
def seperateFrames(video_path, label, labels):
    vidcap = cv2.VideoCapture(video_path)
    correct = 0
    count = 0
    k = 0
    success = True
    imList = list()
    # print vidcap
    while success:
        
        success, image = vidcap.read()
        if labels[count] == label:
          imList.append(image)
          
          
    return imList
  

In [0]:
# people = ['Sadhguru', 'Raman', 'AtulKhatri', 'SaurabhPant', 'Shailendra', 'Sandeep', 'Noise']
# for i in range(0, 7):
#   print 'starting ',people[i]
#   imLis = seperateFrames('test_vid_2.avi', i, y_list)
#   imLis = np.array(imLis)
#   save_string = people[i]+'test.npy'
#   np.save(save_string, imLis)
#   uploadFileToDrive(save_string)

In [0]:
image_shape = (300, 250)
def test2(video_path, labels):
    vidcap = cv2.VideoCapture(video_path)
    correct = 0
    count = 0
    k = 0
    success = True
    imLis = list()
    # print vidcap
    y_labls = list()
    while success:
        
        success, image = vidcap.read()
        count += 1
        if count % 200 == 0:
          break
        img = image
        cv2.imwrite('frame1.jpg', img)
        im = lightnet.Image.from_bytes(open('frame1.jpg', 'rb').read())
        
        boxes = detectPerson(im)
        
        if len(boxes) > 1:
          imLis.append(cv2.resize(img, (150, 120)))
          y_labls.append(labels[count-1])
          continue
        else:
          if len(boxes) == 1:
            imageList = cropImages(boxes, img)
            if len(imageList) == 0:
              imLis.append(cv2.resize(img, (150, 120)))
            else:
              imLis = imLis + imageList
            y_labls.append(labels[count-1])
          else:
            imLis.append(cv2.resize(img, (150, 120)))
            y_labls.append(labels[count-1])
          continue
            
    acc = float(correct)/count
    print 'acc : ', acc
    return imLis, y_labls

In [22]:
imLis, y_labls = test2('test_vid_1.avi', y_list)

acc :  0.0


In [24]:
print len(imLis), len(y_labls)

199 199


In [0]:
# imLis = np.array(imLis, dtype=np.float32)/255.0
np.save('labls1.npy', y_labls)
np.save('imLis1.npy', imLis)
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file = drive.CreateFile()
file.SetContentFile('labls1'+'.npy')
file.Upload()
file2 = drive.CreateFile()
file2.SetContentFile('imLis1'+'.npy')
file2.Upload()